In [2]:
from warcio.archiveiterator import ArchiveIterator
import json 
import requests
import gzip
import shutil
import os

In [ ]:
with open(r"C:\Users\tanbi\partial.warc", 'rb') as stream:
    for record in ArchiveIterator(stream):
        if(record.rec_type == 'response'):
            payload = record.content_stream().read()
            try:
                text = payload.decode('utf-8')
                print(text)
            except UnicodeDecodeError:
                continue
# well i think that downloaded warc is a robots.txt wherease we need plain/html raw data


#This is how the filename for a website stored

urlkey        : com,0--1)/robots.txt
timestamp     : 20230206145812
url           : http://0--1.com/robots.txt
mime          : text/plain
mime-detected : text/x-robots
status        : 200
digest        : LO6DKJEEKTDOQTJNAJFM2ZW5IMJJUBXI
length        : 715
offset        : 897
filename      : crawl-data/CC-MAIN-2023-06/segments/1674764500356.92/robotstxt/CC-MAIN-2023020
                6145603-20230206175603-00085.warc.gz


In [3]:
def download_partial_warc(json_line, base_url="https://data.commoncrawl.org/"):
    
    record = json.loads(json_line)
    # record = json_line
    filename = record["filename"]
    offset = int(record["offset"])
    length = int(record["length"])
    range_header = {"Range": f"bytes={offset}-{offset + length - 1}"}

    print(f"Downloading {length} bytes of data")
    # download partial warc.gz data
    url = base_url + filename
    response = requests.get(url, headers=range_header, stream=True)
    response.raise_for_status()

    # saving compressed file
    partial_gz_filename = "partial.warc.gz"
    with open(partial_gz_filename, "wb") as f:
        for chunk in response.iter_content(chunk_size=8192):
            if chunk:
                f.write(chunk)
    
    # decompress
    partial_warc_filename = "partial.warc"
    with gzip.open(partial_gz_filename, "rb") as f_in:
        with open(partial_warc_filename, "wb") as f_out:
            shutil.copyfileobj(f_in, f_out)
    
    return partial_warc_filename

def extract_text(filename):
    # filepath = os.path.join(r"C:\Users\tanbi", filename)
    with open(filename, 'rb') as stream:
        with open("data1.txt", 'w', encoding='utf-8') as op_file:
            for record in ArchiveIterator(stream):
                if(record.rec_type == 'response'):
                    payload = record.content_stream().read()
                    try:
                        text = payload.decode('utf-8')
                        print(text)
                        op_file.write(text + "\n\n")
                    except UnicodeDecodeError:
                        continue

In [70]:
def fetch_filtered_cc_records(n=399, total_limit=400, index="CC-MAIN-2025-13"):
    api_url = f"http://index.commoncrawl.org/{index}-index"
    params={
        "url": "*.org",
        "output": "json",
        "limit": total_limit
    }

    response = requests.get(api_url, params=params)
    if response.status_code != 200:
        print(response.content)
        print("error ho gaya!")
        return []

    lines = response.text.strip().split('\n')
    results=[]
    for line in lines:
        try:
            record=json.loads(line)
            url = record.get("url", "")
            mime = record.get("mime", "")
            print(f"checking url: {url} | MIME: {mime}")

            if "robots.txt" not in url.lower() and mime == "text/html":
                results.append(record)
                if len(results) > n:
                    break
        except json.JSONDecodeError as e:
            print("err")
            print(e)
    return results

In [71]:
results = fetch_filtered_cc_records()

checking url: http://0.org/ | MIME: text/html
checking url: http://orrca2.0.org/ | MIME: text/html
checking url: http://xorg.0.org/ | MIME: text/html
checking url: https://0--0.org/ | MIME: text/html
checking url: http://0--0.org/ | MIME: text/html
checking url: https://0--0.org/2011/02/ | MIME: text/html
checking url: https://0--0.org/2011/02/13/recumbent/ | MIME: text/html
checking url: https://0--0.org/2018/03/ | MIME: text/html
checking url: https://0--0.org/robots.txt | MIME: text/plain
checking url: http://0--0.org/robots.txt | MIME: text/plain
checking url: https://www.0-0-2.org/ | MIME: text/html
checking url: https://www.0-0-2.org/home | MIME: text/html
checking url: https://www.0-0-2.org/home/become-a-member | MIME: text/html
checking url: https://www.0-0-2.org/robots.txt | MIME: text/html
checking url: http://0-1.org/robots.txt | MIME: unk
checking url: http://0-1-2.org/robots.txt | MIME: text/html
checking url: https://www.0-4.org/2-letter-com-domain-names.html | MIME: text

In [61]:
results[10]

{'urlkey': 'edu,4cd)/about/colleges.html',
 'timestamp': '20250317185354',
 'url': 'https://www.4cd.edu/about/colleges.html',
 'mime': 'text/html',
 'mime-detected': 'application/xhtml+xml',
 'status': '200',
 'digest': 'SHOSGH5MGCKCLRXQZTYXUWFWIKPIPCWR',
 'length': '5107',
 'offset': '618174145',
 'filename': 'crawl-data/CC-MAIN-2025-13/segments/1742004433031.79/warc/CC-MAIN-20250317164600-20250317194600-00826.warc.gz',
 'languages': 'eng',
 'encoding': 'UTF-8'}

In [62]:
# filename = results[0]["filename"]
filepath_warc = download_partial_warc(json.dumps(results[10]))

In [65]:
html_txt = extract_text(filepath_warc)

<!DOCTYPE html>
<html xmlns="http://www.w3.org/1999/xhtml" lang="en">
	<head>
		
    

    
    <script async src="https://www.googletagmanager.com/gtag/js?id=G-0JFEC9MCG9"></script>
    <script>
        window.dataLayer = window.dataLayer || [];
        function gtag(){dataLayer.push(arguments);}
        gtag('js', new Date());
        gtag('config', 'G-0JFEC9MCG9');
    </script>
    

    

		<title>
			Contra Costa Community College District
		</title>
		<meta charset="utf-8"/>
		<meta content="width=device-width, initial-scale=1" name="viewport"/>
		<!-- ***** SHORTCUT ICON ***** -->
		<link href="../_files/images/ico/ccccdfavicon.ico" rel="icon"/>
		<!-- ********************** -->
		
    
	<link href="https://www.w3schools.com/w3css/4/w3.css" rel="stylesheet"/>
	<link href="https://maxcdn.bootstrapcdn.com/bootstrap/3.4.1/css/bootstrap.min.css" rel="stylesheet"/>
	<script src="https://ajax.googleapis.com/ajax/libs/jquery/3.4.1/jquery.min.js" type="text/javascript"></script>
	<scri

In [64]:
# def extract_html_text():
#     results = fetch_filtered_cc_records()

#     for result in results:
#         filename_out = download_partial_warc(result)
from bs4 import BeautifulSoup
def extract_clean_text(html):
    """
    Extracts main readable text from an HTML page.
    Removes scripts, styles, navs, footers, etc.
    """
    soup = BeautifulSoup(html, "html.parser")

    # Remove script, style, header, footer, nav, aside tags
    for tag in soup(["script", "style", "header", "footer", "nav", "aside", "noscript"]):
        tag.decompose()

    # Get raw text
    raw_text = soup.get_text(separator="\n")

    # Clean extra whitespace and blank lines
    lines = [line.strip() for line in raw_text.splitlines()]
    lines = [line for line in lines if line]  # remove empty lines

    clean_text = "\n".join(lines)

    # Optional: remove very short lines (noise)
    clean_text = "\n".join([line for line in clean_text.splitlines() if len(line) > 40])

    return clean_text


In [67]:
with open("data1.txt", "r", encoding="utf-8") as f:
    html_content = f.read()

clean_text = extract_clean_text(html_content)
print(clean_text)

has provided exemplary educational services to hundreds of thousands of residents from the greater West County area. The college has superior faculty and staff members and a learning environment that maximizes the potential of students. Technology is available throughout the institution and connects students to the world to ensure a relevant exchange of ideas. The college is proud of its diverse student body and commitment to individual student success. Excellent programs such as the Center for Science Excellence, The Advocate newspaper, the green Automotive Services program, the Nursing program, and the Culinary Arts program are known throughout the state and the nation. A model of excellence, CCC prides itself on being one of the finest community colleges in the country.
has distinguished itself as one of the nation's most successful community colleges by offering incomparable transfer opportunities and exceptional career-technical programs and degrees to a diverse student body, whil